In [7]:
import sympy as sp
from sympy import Derivative as D
from utils.symbolic import (infinitesimals_generator, higher_infinitesimals_generator,
                            group_operator, der_relabel, deriv_infts, subs_new_vars)
from utils.combinatorics import list_combinatorics

In [8]:
z, t, c = sp.symbols('z t c')
list_combinatorics([z, t, c], 2)

[[z], [t], [c], [z, z], [z, t], [z, c], [t, t], [t, c], [c, c]]

In [9]:
x, t = sp.symbols('x t')
u = sp.Function('u')(x, t)

In [10]:
list_indep = [x, t]
list_dep = [u]
F = - D(u, x, x) + D(u, t)

infts = infinitesimals_generator(list_indep, list_dep)
n_indep = len(list_indep)
n_dep = len(list_dep)
infts_ind = infts[0:n_indep]
infts_dep = infts[n_indep:(n_dep + n_indep )]
inft_derivatives, dep_vars_derivatives = higher_infinitesimals_generator(infts_ind, infts_dep,
                                                                            2, list_indep, list_dep)
infts_dummy = [item for sublist in inft_derivatives for item in sublist]
dep_vars_derivatives = [item for sublist in dep_vars_derivatives for item in sublist]

infts = infts + infts_dummy

F, deriv_names = der_relabel(dep_vars_derivatives, F)

vars_and_derivatives = list_indep + list_dep + deriv_names

XF = group_operator(F, vars_and_derivatives, infts)

XF = subs_new_vars(dep_vars_derivatives, deriv_names, XF)

# A = deriv_infts(infts_ind + infts_dep, list_indep + list_dep, 2)

# XF, deriv_infts_names = der_relabel(A, XF)

# XF = subs_new_vars(deriv_infts_names, A, XF)

# XF = subs_new_vars(deriv_names, dep_vars_derivatives, XF)

In [11]:
XF.expand()

Derivative(eta^u(x, t, u(x, t)), u(x, t))*Derivative(u(x, t), t) - Derivative(eta^u(x, t, u(x, t)), u(x, t))*Derivative(u(x, t), (x, 2)) - Derivative(eta^u(x, t, u(x, t)), (u(x, t), 2))*Derivative(u(x, t), x)**2 - Derivative(u(x, t), t)**2*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + Derivative(u(x, t), t)*Derivative(u(x, t), x)**2*Derivative(xi^t(x, t, u(x, t)), (u(x, t), 2)) - Derivative(u(x, t), t)*Derivative(u(x, t), x)*Derivative(xi^x(x, t, u(x, t)), u(x, t)) + Derivative(u(x, t), t)*Derivative(u(x, t), x)*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), _xi_1, u(x, t)), _xi_1, x) + Derivative(u(x, t), t)*Derivative(u(x, t), x)*Subs(Derivative(xi^t(x, t, _xi), _xi, x), _xi, u(x, t)) + Derivative(u(x, t), t)*Derivative(u(x, t), (x, 2))*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + Derivative(u(x, t), t)*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), (_xi_1, 2)), _xi_1, x) - Derivative(u(x, t), t)*Subs(Derivative(xi^t(x, _xi_2, u(x, t)), _xi_2), _xi_2, t) + Derivative(u(x, t), x)**3*Derivative(xi^x(x,

In [5]:
infts[0]

xi^x(x, t, u(x, t))

In [6]:
infts[1]

xi^t(x, t, u(x, t))

In [7]:
infts[4]

-(Derivative(u(x, t), t)*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + Derivative(xi^t(x, t, u(x, t)), t))*Derivative(u(x, t), t) - (Derivative(u(x, t), t)*Derivative(xi^x(x, t, u(x, t)), u(x, t)) + Derivative(xi^x(x, t, u(x, t)), t))*Derivative(u(x, t), x) + Derivative(eta^u(x, t, u(x, t)), t) + Derivative(eta^u(x, t, u(x, t)), u(x, t))*Derivative(u(x, t), t)

In [8]:
infts[5]

(2*Derivative(eta^u(x, t, u(x, t)), u(x, t)) - 2*Derivative(u(x, t), t)*Derivative(xi^t(x, t, u(x, t)), u(x, t)) - 4*Derivative(u(x, t), x)*Derivative(xi^x(x, t, u(x, t)), u(x, t)) - Subs(Derivative(xi^x(_xi_1, t, u(x, t)), _xi_1), _xi_1, x))*Derivative(u(x, t), (x, 2)) + Derivative(-(Derivative(u(x, t), x)*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + Derivative(xi^t(x, t, u(x, t)), x))*Derivative(u(x, t), t) - (Derivative(u(x, t), x)*Derivative(xi^x(x, t, u(x, t)), u(x, t)) + Derivative(xi^x(x, t, u(x, t)), x))*Derivative(u(x, t), x) + Derivative(eta^u(x, t, u(x, t)), x) + Derivative(eta^u(x, t, u(x, t)), u(x, t))*Derivative(u(x, t), x), x) - Derivative(u(x, t), (x, 2))*Derivative(xi^x(x, t, u(x, t)), x) - Derivative(xi^t(x, t, u(x, t)), x)*Derivative(u(x, t), x, t)

In [11]:
infts[6]

(2*Derivative(eta^u(x, t, u(x, t)), u(x, t)) - 2*Derivative(u(x, t), t)*Derivative(xi^t(x, t, u(x, t)), u(x, t)) - 4*Derivative(u(x, t), x)*Derivative(xi^x(x, t, u(x, t)), u(x, t)) - Subs(Derivative(xi^x(_xi_1, t, u(x, t)), _xi_1), _xi_1, x))*Derivative(u(x, t), t, x) + Derivative(-(Derivative(u(x, t), x)*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + Derivative(xi^t(x, t, u(x, t)), x))*Derivative(u(x, t), t) - (Derivative(u(x, t), x)*Derivative(xi^x(x, t, u(x, t)), u(x, t)) + Derivative(xi^x(x, t, u(x, t)), x))*Derivative(u(x, t), x) + Derivative(eta^u(x, t, u(x, t)), x) + Derivative(eta^u(x, t, u(x, t)), u(x, t))*Derivative(u(x, t), x), t) - Derivative(u(x, t), (x, 2))*Derivative(xi^x(x, t, u(x, t)), t) - Derivative(xi^t(x, t, u(x, t)), t)*Derivative(u(x, t), x, t)

In [7]:
XF.expand()

2*Derivative(eta^u(x, t, u(x, t)), u(x, t))*Derivative(u(x, t), t) - 4*Derivative(eta^u(x, t, u(x, t)), u(x, t))*Derivative(u(x, t), (x, 2)) - 2*Derivative(eta^u(x, t, u(x, t)), (u(x, t), 2))*Derivative(u(x, t), x)**2 - 2*Derivative(u(x, t), t)**2*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + 2*Derivative(u(x, t), t)*Derivative(u(x, t), x)**2*Derivative(xi^t(x, t, u(x, t)), (u(x, t), 2)) - 2*Derivative(u(x, t), t)*Derivative(u(x, t), x)*Derivative(xi^x(x, t, u(x, t)), u(x, t)) + Derivative(u(x, t), t)*Derivative(u(x, t), x)*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), _xi_1, u(x, t)), _xi_1, x) + 2*Derivative(u(x, t), t)*Derivative(u(x, t), x)*Subs(Derivative(xi^t(x, t, _xi), _xi, x), _xi, u(x, t)) + 4*Derivative(u(x, t), t)*Derivative(u(x, t), (x, 2))*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + Derivative(u(x, t), t)*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), (_xi_1, 2)), _xi_1, x) - Derivative(u(x, t), t)*Subs(Derivative(xi^t(x, _xi_2, u(x, t)), _xi_2), _xi_2, t) + 2*Derivative(u(x, t), x)**3*

In [22]:
XF

-(Derivative(eta^u(x, t, u(x, t)), (u(x, t), 2))*Derivative(u(x, t), x) + Subs(Derivative(eta^u(x, t, _xi), _xi, x), _xi, u(x, t)))*Derivative(u(x, t), x) - (Derivative(u(x, t), t)*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + Subs(Derivative(xi^t(x, _xi_2, u(x, t)), _xi_2), _xi_2, t))*Derivative(u(x, t), t) - (Derivative(u(x, t), t)*Derivative(xi^x(x, t, u(x, t)), u(x, t)) + Subs(Derivative(xi^x(x, _xi_2, u(x, t)), _xi_2), _xi_2, t))*Derivative(u(x, t), x) + 2*(Derivative(u(x, t), x)*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + Subs(Derivative(xi^t(_xi_1, t, u(x, t)), _xi_1), _xi_1, x))*Derivative(u(x, t), t, x) + 2*(Derivative(u(x, t), x)*Derivative(xi^x(x, t, u(x, t)), u(x, t)) + Subs(Derivative(xi^x(_xi_1, t, u(x, t)), _xi_1), _xi_1, x))*Derivative(u(x, t), (x, 2)) + ((Derivative(u(x, t), x)*Derivative(xi^t(x, t, u(x, t)), (u(x, t), 2)) + Subs(Derivative(xi^t(x, t, _xi), _xi, x), _xi, u(x, t)))*Derivative(u(x, t), x) + Derivative(u(x, t), x)*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), _xi_

In [59]:
dir(infts_ind[1].diff(x).args[-1])

['__abs__',
 '__add__',
 '__class__',
 '__complex__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__round__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__sympy__',
 '__truediv__',
 '__trunc__',
 '__weakref__',
 '_add_handler',
 '_args',
 '_assumptions',
 '_compare_pretty',
 '_constructor_postprocessor_mapping',
 '_diff_wrt',
 '_eval_adjoint',
 '_eval_as_leading_term',
 '_eval_conjugate',
 '_eval_derivative',
 '_eval_derivativ

In [40]:
deriv_infts_names[19]

xi^t_tt

In [11]:
XF.subs({sp.simplify(A[19]):deriv_infts_names[19]}).expand()

eta^u_u*u_t - eta^u_u*u_xx - eta^u_uu*u_x**2 - u_t**2*xi^t_u + u_t*u_x**2*xi^t_uu - u_t*u_x*xi^x_u + u_t*u_x*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), _xi_1, u(x, t)), _xi_1, x) + u_t*u_x*Subs(Derivative(xi^t(x, t, _xi), _xi, x), _xi, u(x, t)) + u_t*u_xx*xi^t_u + u_t*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), (_xi_1, 2)), _xi_1, x) - u_t*Subs(Derivative(xi^t(x, _xi_2, u(x, t)), _xi_2), _xi_2, t) + u_x**3*xi^x_uu + u_x**2*Subs(Derivative(xi^x(_xi_1, t, u(x, t)), _xi_1, u(x, t)), _xi_1, x) + u_x**2*Subs(Derivative(xi^x(x, t, _xi), _xi, x), _xi, u(x, t)) + 2*u_x*u_xt*xi^t_u + 3*u_x*u_xx*xi^x_u + u_x*Subs(Derivative(xi^x(_xi_1, t, u(x, t)), (_xi_1, 2)), _xi_1, x) - u_x*Subs(Derivative(xi^x(x, _xi_2, u(x, t)), _xi_2), _xi_2, t) - u_x*Subs(Derivative(eta^u(_xi_1, t, u(x, t)), _xi_1, u(x, t)), _xi_1, x) - u_x*Subs(Derivative(eta^u(x, t, _xi), _xi, x), _xi, u(x, t)) + 2*u_xt*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), _xi_1), _xi_1, x) + 2*u_xx*Subs(Derivative(xi^x(_xi_1, t, u(x, t)), _xi_1), _xi_1,

In [20]:
d = A[6]
d_str = str(d.args[0]).split('(')[0] + '_' 
d_order = list(d.args)
d_order.pop(0)
d_order[0][0]

u(x, t)

In [7]:
deriv_infts_names

[xi^x_x,
 xi^t_x,
 eta^u_x,
 xi^x_t,
 xi^t_t,
 eta^u_t,
 (xi^x_u(x, t)),
 (xi^t_u(x, t)),
 (eta^u_u(x, t)),
 xi^x_xx,
 xi^t_xx,
 eta^u_xx,
 xi^x_xt,
 xi^t_xt,
 eta^u_xt,
 (xi^x_xu(x, t)),
 (xi^t_xu(x, t)),
 (eta^u_xu(x, t)),
 xi^x_tt,
 xi^t_tt,
 eta^u_tt,
 (xi^x_tu(x, t)),
 (xi^t_tu(x, t)),
 (eta^u_tu(x, t)),
 (xi^x_u(x, t)u(x, t)),
 (xi^t_u(x, t)u(x, t)),
 (eta^u_u(x, t)u(x, t))]

In [12]:
(- D(u, x, x) + D(u, t)).make_args(- D(u, x, x) + D(u, t))

(-Derivative(u(x, t), (x, 2)), Derivative(u(x, t), t))

In [6]:
D(u,t)

Derivative(u(x, t), t)

In [5]:
sp.simplify(XF.expand())

Derivative(eta^u(x, t, u(x, t)), u(x, t))*Derivative(u(x, t), t) - Derivative(eta^u(x, t, u(x, t)), u(x, t))*Derivative(u(x, t), (x, 2)) - Derivative(eta^u(x, t, u(x, t)), (u(x, t), 2))*Derivative(u(x, t), x)**2 - Derivative(u(x, t), t)**2*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + Derivative(u(x, t), t)*Derivative(u(x, t), x)**2*Derivative(xi^t(x, t, u(x, t)), (u(x, t), 2)) - Derivative(u(x, t), t)*Derivative(u(x, t), x)*Derivative(xi^x(x, t, u(x, t)), u(x, t)) + Derivative(u(x, t), t)*Derivative(u(x, t), x)*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), _xi_1, u(x, t)), _xi_1, x) + Derivative(u(x, t), t)*Derivative(u(x, t), x)*Subs(Derivative(xi^t(x, t, _xi), _xi, x), _xi, u(x, t)) + Derivative(u(x, t), t)*Derivative(u(x, t), (x, 2))*Derivative(xi^t(x, t, u(x, t)), u(x, t)) + Derivative(u(x, t), t)*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), (_xi_1, 2)), _xi_1, x) - Derivative(u(x, t), t)*Subs(Derivative(xi^t(x, _xi_2, u(x, t)), _xi_2), _xi_2, t) + Derivative(u(x, t), x)**3*Derivative(xi^x(x,

In [6]:
infts[3].expand()

Derivative(eta^u(x, t, u(x, t)), u(x, t))*Derivative(u(x, t), x) - Derivative(u(x, t), t)*Derivative(u(x, t), x)*Derivative(xi^t(x, t, u(x, t)), u(x, t)) - Derivative(u(x, t), t)*Subs(Derivative(xi^t(_xi_1, t, u(x, t)), _xi_1), _xi_1, x) - Derivative(u(x, t), x)**2*Derivative(xi^x(x, t, u(x, t)), u(x, t)) - Derivative(u(x, t), x)*Subs(Derivative(xi^x(_xi_1, t, u(x, t)), _xi_1), _xi_1, x) + Subs(Derivative(eta^u(_xi_1, t, u(x, t)), _xi_1), _xi_1, x)

In [8]:
y.args

(x,)

In [50]:
A = D(psi, x,x,t,z,z,x)
dir(A)

['__abs__',
 '__add__',
 '__class__',
 '__complex__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__round__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__sympy__',
 '__truediv__',
 '__trunc__',
 '__weakref__',
 '_add_handler',
 '_args',
 '_assumptions',
 '_compare_pretty',
 '_constructor_postprocessor_mapping',
 '_diff_wrt',
 '_dispatch_eval_derivative_n_times',
 '_eval_adjoint',
 '_eval_as_leading_term',
 '_eval_conjugate',


In [51]:
B = list(A.args)
B.pop(0)
B

[(x, 2), (t, 1), (z, 2), (x, 1)]

In [31]:

y_xx = sp.symbols('y_xx')
F.subs(D(y,x,x), y_xx)


y_xx

In [12]:
F = D(y, x, x)

XF = group_operator(F , vars_and_derivatives, infts)

> c:\users\danit\documents\research\lie_symmetry_analysis_agorithm\notebooks\utils\symbolic.py(206)group_operator()
    205     import ipdb; ipdb.set_trace()
--> 206     for var, inft in var_inft:
    207         LF += inft*D(F, var)

ipdb> n
> c:\users\danit\documents\research\lie_symmetry_analysis_agorithm\notebooks\utils\symbolic.py(207)group_operator()
    206     for var, inft in var_inft:
--> 207         LF += inft*D(F, var)
    208     return LF

ipdb> LF
0
ipdb> n
> c:\users\danit\documents\research\lie_symmetry_analysis_agorithm\notebooks\utils\symbolic.py(206)group_operator()
    205     import ipdb; ipdb.set_trace()
--> 206     for var, inft in var_inft:
    207         LF += inft*D(F, var)

ipdb> LF
xi^x(x, y(x))*Derivative(y(x), (x, 3))
ipdb> n
> c:\users\danit\documents\research\lie_symmetry_analysis_agorithm\notebooks\utils\symbolic.py(207)group_operator()
    206     for var, inft in var_inft:
--> 207         LF += inft*D(F, var)
    208     return LF

ipdb> n
> c:\user

ipdb> n
--Return--
None
> <ipython-input-12-620d96db3cdf>(3)<module>()
      1 F = D(y, x, x)
      2 
----> 3 XF = group_operator(F , vars_and_derivatives, infts)

ipdb> q


BdbQuit: 

In [10]:
u = sp.Function('u')(x,y)

In [58]:
str(D(u, x, x, y).args[0]).split('(')[0]

'u'

In [12]:
XF.subs({D(y, x, x):0})

(-(Derivative(xi^x(x, y(x)), y(x))*Derivative(y(x), x) + Subs(Derivative(xi^x(_xi_1, y(x)), _xi_1), _xi_1, x))*Derivative(y(x), x) + Derivative(eta^y(x, y(x)), y(x))*Derivative(y(x), x) + Subs(Derivative(eta^y(_xi_1, y(x)), _xi_1), _xi_1, x))*Derivative(0, Derivative(y(x), x)) + ((Derivative(eta^y(x, y(x)), (y(x), 2))*Derivative(y(x), x) + Subs(Derivative(eta^y(x, _xi), _xi, x), _xi, y(x)))*Derivative(y(x), x) + (-(Derivative(xi^x(x, y(x)), (y(x), 2))*Derivative(y(x), x) + Subs(Derivative(xi^x(x, _xi), _xi, x), _xi, y(x)))*Derivative(y(x), x) - Derivative(y(x), x)*Subs(Derivative(xi^x(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x) - Subs(Derivative(xi^x(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x))*Derivative(y(x), x) + Derivative(y(x), x)*Subs(Derivative(eta^y(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x) + Subs(Derivative(eta^y(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x))*Subs(Derivative(y(x), (x, 2), _xi), _xi, 0) + eta^y(x, y(x))*Derivative(0, y(x)) + xi^x(x, y(x))*Derivative(0, x)

In [ ]:
def group_operator(F, variables, infts):
    var_inft = zip(variables, infts)
    LF = 0
    for var, inft in var_inft:
        LF += inft*D(L, var)

In [15]:
sp.collect(inft_derivatives[2][0].expand(), y)

(Derivative(eta^y(x, y(x)), y(x)) - Subs(Derivative(xi^x(_xi_1, y(x)), _xi_1), _xi_1, x))*Derivative(y(x), (x, 3)) + (Derivative(eta^y(x, y(x)), (y(x), 3)) - Subs(Derivative(xi^x(_xi_1, y(x)), _xi_1, (y(x), 2)), _xi_1, x) - 2*Subs(Derivative(xi^x(x, _xi), (_xi, 2), x), _xi, y(x)))*Derivative(y(x), x)**3 + (-5*Derivative(xi^x(x, y(x)), (y(x), 2))*Derivative(y(x), x)**2 - 2*Subs(Derivative(xi^x(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x) + Subs(Derivative(eta^y(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x) + 2*Subs(Derivative(eta^y(x, _xi), _xi, x), _xi, y(x)))*Derivative(y(x), (x, 2)) + (Subs(Derivative(eta^y(_xi_1, y(x)), _xi_1, (y(x), 2)), _xi_1, x) + 2*Subs(Derivative(eta^y(x, _xi), (_xi, 2), x), _xi, y(x)) - 2*Subs(Derivative(xi^x(_xi_1, y(x)), (_xi_1, 2), y(x)), _xi_1, x) - Subs(Derivative(xi^x(x, _xi), _xi, (x, 2)), _xi, y(x)))*Derivative(y(x), x)**2 + (3*Derivative(eta^y(x, y(x)), (y(x), 2))*Derivative(y(x), (x, 2)) - 2*Derivative(xi^x(x, y(x)), y(x))*Derivative(y(x), (x, 3)) - 3*Derivative(y(x

In [6]:
dep_vars_derivatives[0][0]


Derivative(y(x), x)

In [8]:
for i in range(2):
    for j in range(2):
        print(i, j)

0 0
0 1
1 0
1 1


In [8]:
list_indep = [x, t]
list_dep = [u]
infts = infinitesimals_generator(list_indep, list_dep)

In [11]:
infts*2

[xi^x(x, t, psi(x, t)),
 xi^t(x, t, psi(x, t)),
 eta^psi(x, t, psi(x, t)),
 xi^x(x, t, psi(x, t)),
 xi^t(x, t, psi(x, t)),
 eta^psi(x, t, psi(x, t))]

In [7]:
def infinitesimals_generator(list_indep, list_dep):
    infts = []
    variables = list_indep + list_dep
    for var in list_indep:
        f = f'xi^{var}'
        infts.append(sp.Function(f)(*variables))
    for var in list_dep:
        f = f'eta^{var}'.split('(')[0]
        infts.append(sp.Function(f)(*variables))
    return infts

In [107]:
xi_x = Function('xi^x')(x, t, u)
xi_t = Function('xi^x')(x, t, u)
eta_u = Function('eta^u')(x, t, u)

[x, t]

In [116]:
list_inf = [xi_x, xi_t, eta_u]

In [115]:
xi_x

xi^x(x, t, u(x, t))

In [124]:
x, y, z, t, tau = symbols('x y z t tau')

In [126]:
comb = cb.list_combinatorics([x,y,z,y,tau])

In [130]:
A = [1,2,3,4,5]
A[-1]

5

In [ ]:
def group_operator(F, list_indep, list_dep, list_inft):
    list_var = list_indep + list_dep
    var_inft = zip(list_var, list_inft)
    LF = 0
    for var, inft in var_inft:
        LF += inft*D(L, var)

def higher_infinitesimals_generator(list_inft_indep ,list_inft, Order, list_indep, list_dep):
    dep_vars_derivatives = []
    inft_derivatives = []
    var_combinatorics = cb.list_combinatorics(variables)
    depvars_etas = zip(list_dep, list_inft)
    indvar_xis =  zip(list_indep, list_inft_indep)
    for ind_vars in var_combinatorics:
        for dep_var, eta in depvar_etas: 
        if len(ind_vars) == 1:
            dep_var_derivatives.append(D(dep_var,ind_var[0]))
        else:
            idx = var_combinatorics.index(ind_vars[:-1])   
            dep_vars_derivatives.append(
                D(dep_vars_derivatives[idx],ind_vars[-1])
            dummy = inft_derivatives[idx].diff(ind_vars[-1])
            for ind_var, xi in indvar_xis:
                dummy += D(dep_vars_derivatives[idx], ind_var)*(
                    xi.diff(ind_vars[-1]))
            inft_derivatives.append(dummy)
    return
            

In [3]:
def Infinitesimals_Generator(N, q, p, y, x):
    """This function creates the infinitesimals for the
       N-th order derivative of y and agroup them in a list.
       
       Args:
       N (int):      the order of the highest infinitesimal
       q (list):     inifinitesimal of the dependent variable y
       p (Function): infinitesimal of x
       y (function): depedant variable
       x (symbol):   the independent variable
    """
    n  = [q]
    Dp = p.diff(x)
    for i in range(1,N+1):
        n.append(expand(n[i-1].diff(x)-y.diff(x,i)*Dp))
    return n
def Group_Operator(F, n, p, y, x, X):
    """This function creates the infinitesimals for the
       N-th order derivative of y and agroup them in a list.
       
       Args:
       F (add):      differential equation equated to zero. All
                     x terms should be written in terms of a dummy
                     variable X, All dependant variables and their 
                     derivatives should be written in terms of the 
                     independent variable x
       n (list):     list containing a function of (x,y). It is 
                     the infinitesimal of y.
       p (Function): it is the infinitesimal of x
       y (function): depedant variable
       x (symbol):   the independent variable
       X (symbol):   dummy variable to take partial derivatives
    """
    XF = p*F.diff(X)
    for i in range(0, len(n)):
        XF += n[i]*F.diff(y.diff(x,i))
    return expand(XF)

In [4]:
F = y_xx

In [5]:
n = Infinitesimals_Generator(2, eta, xi, y, x)
XF = Group_Operator(F, n, xi, y, x, X)

In [8]:
XF = collect(XF,y)

In [15]:
XF

(Derivative(eta(x, y(x)), y(x)) - 2*Subs(Derivative(xi(_xi_1, y(x)), _xi_1), _xi_1, x))*Derivative(y(x), (x, 2)) + (Derivative(eta(x, y(x)), (y(x), 2)) - Subs(Derivative(xi(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x) - Subs(Derivative(xi(x, _xi), _xi, x), _xi, y(x)))*Derivative(y(x), x)**2 + (-3*Derivative(xi(x, y(x)), y(x))*Derivative(y(x), (x, 2)) - Subs(Derivative(xi(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x) + Subs(Derivative(eta(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x) + Subs(Derivative(eta(x, _xi), _xi, x), _xi, y(x)))*Derivative(y(x), x) - Derivative(xi(x, y(x)), (y(x), 2))*Derivative(y(x), x)**3 + Subs(Derivative(eta(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x)

In [26]:
(XF.make_args(XF)[2]).make_args(XF.make_args(XF)[2])[0]

-3*Derivative(xi(x, y(x)), y(x))*Derivative(y(x), (x, 2)) - Subs(Derivative(xi(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x) + Subs(Derivative(eta(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x) + Subs(Derivative(eta(x, _xi), _xi, x), _xi, y(x))

In [9]:
def Group_Det_Eq(XF, N, X, y):
    XF_dummy = collect(XF,var)
    XF_list = list(XF_dummy.make_args(XF_dummy))
    for i in range(len(XF_list)):
        XF_dummy = collect(XF_list[0])
        XF_list[0] = list(XF_dummy[0].make_args)
        
        
    return n_terms, variables

In [10]:
Group_Det_Eq(XF, 2, X, y)

TypeError: cannot determine truth value of Relational

In [28]:
list(XF.make_args(XF))

[Derivative(eta(x, y(x)), (y(x), 2))*Derivative(y(x), x)**2,
 Derivative(eta(x, y(x)), y(x))*Derivative(y(x), (x, 2)),
 Derivative(y(x), x)*Subs(Derivative(eta(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x),
 Derivative(y(x), x)*Subs(Derivative(eta(x, _xi), _xi, x), _xi, y(x)),
 -Derivative(y(x), x)**2*Subs(Derivative(xi(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x),
 -Derivative(y(x), x)**2*Subs(Derivative(xi(x, _xi), _xi, x), _xi, y(x)),
 -Derivative(xi(x, y(x)), (y(x), 2))*Derivative(y(x), x)**3,
 -Derivative(y(x), x)*Subs(Derivative(xi(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x),
 -2*Derivative(y(x), (x, 2))*Subs(Derivative(xi(_xi_1, y(x)), _xi_1), _xi_1, x),
 -3*Derivative(xi(x, y(x)), y(x))*Derivative(y(x), x)*Derivative(y(x), (x, 2)),
 Subs(Derivative(eta(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x)]

In [13]:
A = [4,2,6,35,4]
A.index(max(A))

3

In [63]:
i=0
simplify((collect(XF,X).make_args(collect(XF,y))[2]))#.subs(y,x^2/2))

(-3*Derivative(xi(x, y(x)), y(x))*Derivative(y(x), (x, 2)) - Subs(Derivative(xi(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x) + Subs(Derivative(eta(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x) + Subs(Derivative(eta(x, _xi), _xi, x), _xi, y(x)))*Derivative(y(x), x)

In [15]:
A

[4, 2, 6, 4]

In [3]:
y

y(x)

In [4]:
eta

eta(x, y(x))

In [5]:
xi

xi(x, y(x))

In [29]:
(eta*y).kind

AttributeError: 'Mul' object has no attribute 'kind'

In [27]:
eta.kind

AttributeError: 'eta' object has no attribute 'kind'

In [12]:
(y.diff(x)*eta).make_args(y.diff(x)*eta)

(Derivative(y(x), x), eta(x, y(x)))

In [16]:
type(integrate(y.diff(x),x))

y

In [31]:
integrate(eta.diff(y,x,2),x)

KeyboardInterrupt: 

In [40]:
parse_derivative(y.diff(x,3))

(y(x), (x, 3))

In [37]:
eta.diff(x)-y_x*eta.diff(y)

Subs(Derivative(eta(_xi_1, y(x)), _xi_1), _xi_1, x)

In [30]:
def parse_derivative(deriv):
    # scan derivatives tower in the input expression and return
    # underlying function and maximal differentiation order
    expr, sym, order = deriv.expr, deriv.variables[0], 1

    for s in deriv.variables[1:]:
        if s == sym:
            order += 1
        else:
            raise NotImplementedError(
                'Improve MV Derivative support in collect')

    while isinstance(expr, Derivative):
        s0 = expr.variables[0]

        for s in expr.variables:
            if s != s0:
                raise NotImplementedError(
                    'Improve MV Derivative support in collect')

        if s0 == sym:
            expr, order = expr.expr, order + len(expr.variables)
        else:
            break

    return expr, (sym, Rational(order))

In [42]:
(eta.diff(x)).variables

AttributeError: 'Add' object has no attribute 'variables'